**Given an undirected graph, return true if and only if it is bipartite.**

**Recall that a graph is bipartite if we can split it's set of nodes into two independent subsets A and B such that every edge in the graph has one node in A and another node in B.**

**The graph is given in the following form: `graph[i]` is a list of indexes `j` for which the edge between nodes `i` and `j` exists.  Each node is an integer between `0` and `graph.length - 1`.  There are no self edges or parallel edges: `graph[i]` does not contain `i`, and it doesn't contain any element twice.**

note: input given is adjList

Bipartite:
- any tree (no cycle) is bipartite -- can alternate every level in different teams
- any graph with odd length cycle is not bipartite 
- any graph with even length cycle is bipartite
- if bfs/dfs tree with no cross edge means no cycles --> bipartite

in bfs: any cross edge will create a cycle -- but is the length of the cycle odd or even? <br>
in dfs: any back  edge will create a cycle -- but is the length of the cycle odd or even? <br>
2 types of cross edges: in same level or adjacent levels<br>
2 types of back  edges: going to same label or complementary label

create bfs/dfs that returns boolean if detect bipartite or not

bfs tree - track by level# (distance array) -- shortest path distance values<br>
- if cross edge which is not parent is in the same level ==> odd length cycle ==> not bipartite
- if cross edge which is not parent goes to node in adjacent level ==> even length cycle ==> continue
- if no odd length cycle in entire graph ==> conclude bipartite

dfs tree - track by alternate colors/ complementary binary label<br>
- if back edge which is not parent goes to node with same label ==> odd length cycle ==> not bipartite
- if back edge which is not parent goes to node with complementary label ==> even length cycle ==> continue
- if no odd length cycle in entire graph ==> conclude bipartite

following template

In [1]:
from collections import deque

def isBipartite(graph):
    adjList = graph  #input given is adjList
    n = len(graph)   #number of vertices

    visited  = [-1] * n
    parent   = [-1] * n
    distance = [-1] * n  #level numbers for bfs tree - shortest path distance values
    color    = [-1] * n  #binary colors for dfs tree
    
    
    def bfs(node):
        visited[node] = 1
        q = deque([node])
        while q:
            node = q.popleft()
            for neighbor in adjList[node]:
                if visited[neighbor] == -1:
                    visited[neighbor] = 1
                    parent[neighbor] = node
                    distance[neighbor] = distance[node] + 1
                    q.append(neighbor)
                else:                            #cross edge
                    if neighbor != parent[node]:
                        if distance[neighbor] == distance[node]: #cross edge same level (odd length cycle) => not bipartite
                            return False    
        return True                                              #no odd length cycle in whole graph => bipartite

    def dfs(node):
        visited[node] = 1
        if parent[node] == -1:                     #set color of parent
            color[node] = 0
        for neighbor in adjList[node]:
            if visited[neighbor] == -1:
                parent[neighbor] = node
                color[neighbor] = not color[node]  #set color of neighbor complementary to its parent color
                if dfs(neighbor) == False:         #indirect detect 
                    return False
            else:
                if color[neighbor] == color[node]: #backedge back to same color = not bipartite (direct detect)
                    return False 
        return True                                #no odd length cycles in whole graph => bipartite
    
    
    for v in range(n):           #check for odd length cycle in all components
        if visited[v] == -1:
            if bfs(v) == False:  #even if one CC not bipartite; then whole graph is not bipartite
                return False
    return True                  #all CC are have no odd cycle ==> bipartite

In [2]:
graph1 = [[1,3], [0,2], [1,3], [0,2]]
graph2 = [[1,2,3], [0,2], [0,1,3], [0,2]]
print(isBipartite(graph1))
print(isBipartite(graph2))

True
False


- cross edge in same level vertices --> odd length cycle --> not bipartite
- cross edge in adjacent level vertices --> even length cycle --> continue
- back edge going to node with same label --> odd length cycle --> not bipartite
- back edge going to node with complementary label --> even length cycle --> continue

assignment of color and dfs traversal in the function dfs - need only color array (don't need extra parent or visited array)

check for all disconnected components of the graph, start by coloring the current node and then in colorDfs, color all neighbors in opposite color from current node. If we find a neighbor colored the same color as the current node, then not bipartite.

dfs recursive

In [3]:
#variant:
def isBipartite(graph):
    color = [-1] * len(graph) #track color and visited
    
    def colorDfs(vertex):                      #color neighbors opposite color from current node
        for neighbor in graph[vertex]:
            if color[neighbor] == -1:          #not visited yet
                color[neighbor] = color[vertex] ^ 1
                if not colorDfs(neighbor):
                    return False
            elif color[neighbor] == color[vertex]: #visited already and same color
                return False
        return True
    
    for vertices in range(len(graph)):         #check for all disconnected components in graph
        if color[vertices] == -1:
            color[vertices] = 0                #start by coloring current node
            if not colorDfs(vertices):
                return False
    return True

iterative dfs

In [4]:
#variant:
def isBipartite(graph):
    color = {}
    for vertex in range(len(graph)):        #check for all disconnected components in graph
        if vertex not in color:
            color[vertex] = 0
            stack = [vertex]
            while stack:
                node = stack.pop()
                for neighbor in graph[node]:
                    if neighbor not in color:
                        color[neighbor] = color[node] ^ 1
                        stack.append(neighbor)
                    else:
                        if color[neighbor] == color[node]:
                            return False
    return True

bfs

In [5]:
#variant:
def isBipartite(graph):
    distance = {} #track levels and visited
    
    def bfs(vertex):
        distance[vertex] = 0
        q = deque([vertex])
        while q:
            node = q.popleft()
            for neighbor in graph[node]:
                if neighbor not in distance:                 #not visited yet
                    distance[neighbor] = distance[node] + 1
                    q.append(neighbor)
                else:
                    if distance[neighbor] == distance[node]: #visited and in same level = odd cycle
                        return False
        return True
    
    for vertices in range(len(graph)):         #check for all disconnected components in graph
        if vertices not in distance:
            if not bfs(vertices):
                return False
    return True